# 📊 Análisis Exploratorio de Datos (EDA) - Comentarios Tóxicos de YouTube

Este notebook contiene el análisis exploratorio de datos para el proyecto de detección de comentarios de odio en YouTube.

## 📋 Objetivos del EDA:
 - Explorar la estructura y calidad de los datos
 - Analizar la distribución de las diferentes categorías de toxicidad
 - Identificar patrones en los comentarios tóxicos vs no tóxicos
 - Detectar posibles desequilibrios en las clases
 - Análisis de texto: longitud, palabras más frecuentes, etc.
 - Identificar insights para el preprocesamiento y modelado

In [ ]:
# Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Carga y Exploración Inicial de los Datos

In [ ]:
df = pd.read_csv('../data/youtoxic_english_1000.csv')

print("🔍 INFORMACIÓN BÁSICA DEL DATASET")
print("="*50)
print(f"📊 Forma del dataset: {df.shape}")
print(f"📝 Número de comentarios: {df.shape[0]:,}")
print(f"🏷️ Número de características: {df.shape[1]}")

In [ ]:
print("\n📋 PRIMERAS 5 FILAS DEL DATASET:")
print("="*50)
df.head()

In [ ]:
print("\n📊 INFORMACIÓN GENERAL:")
print("="*50)
df.info()

In [ ]:
print("\n📈 ESTADÍSTICAS DESCRIPTIVAS:")
print("="*50)
df.describe()

## 2. Análisis de las Etiquetas de Toxicidad

In [ ]:
# Identificar las columnas de etiquetas (excluyendo metadatos y columnas con información insuficiente)
label_columns = [col for col in df.columns if col not in ['CommentId', 'VideoId', 'Text', 'index']]

# Identificar etiquetas con datos insuficientes para análisis
insufficient_labels = []
for col in label_columns:
    true_count = (df[col] == True).sum()
    if true_count <= 1:  # Menos de 2 casos positivos
        insufficient_labels.append(col)

# Filtrar etiquetas con datos suficientes para análisis
analysis_labels = [col for col in label_columns if col not in insufficient_labels]

print(f"🔍 Etiquetas identificadas: {label_columns}")
print(f"🔍 Etiquetas con datos insuficientes (≤1 casos): {insufficient_labels}")
print(f"🔍 Etiquetas para análisis: {analysis_labels}")
print(f"🔍 Total de etiquetas para análisis: {len(analysis_labels)}")

In [ ]:
# Análisis de distribución de cada etiqueta
print("\n📊 DISTRIBUCIÓN DE ETIQUETAS:")
print("="*50)

label_stats = {}
for col in label_columns:
    true_count = (df[col] == True).sum()
    false_count = (df[col] == False).sum()
    true_pct = (true_count / len(df)) * 100
    
    label_stats[col] = {
        'True': true_count,
        'False': false_count,
        'True_pct': true_pct
    }
    
    print(f"{col}:")
    print(f"  ✅ True: {true_count:,} ({true_pct:.2f}%)")
    print(f"  ❌ False: {false_count:,} ({100-true_pct:.2f}%)")
    print()

if insufficient_labels:
    print("⚠️ ETIQUETAS EXCLUIDAS DEL ANÁLISIS:")
    print("="*50)
    for col in insufficient_labels:
        true_count = (df[col] == True).sum()
        print(f"{col}: Solo {true_count} caso(s) positivo(s) - Insuficiente para análisis")


In [ ]:
# Crear un gráfico de barras horizontal para mejor visualización
plt.figure(figsize=(12, 8))

# Preparar datos para el gráfico
labels = []
true_counts = []
percentages = []

for col in label_columns:
    labels.append(col.replace('Is', '').replace('is', ''))
    true_count = (df[col] == True).sum()
    true_counts.append(true_count)
    percentages.append((true_count / len(df)) * 100)

# Crear gráfico de barras horizontal
bars = plt.barh(labels, true_counts, color=plt.cm.Reds(np.linspace(0.3, 0.8, len(labels))))

# Añadir etiquetas con porcentajes
for i, (count, pct) in enumerate(zip(true_counts, percentages)):
    plt.text(count + 5, i, f'{count} ({pct:.1f}%)', 
             va='center', fontweight='bold', fontsize=10)

plt.xlabel('Número de Comentarios Tóxicos', fontsize=12, fontweight='bold')
plt.title('📊 Distribución de Comentarios Tóxicos por Categoría', fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Análisis de Correlaciones entre Etiquetas

In [ ]:
# Matriz de correlación entre etiquetas
correlation_matrix = df[analysis_labels].corr()

# Visualizar matriz de correlación
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title("Correlación entre tipos de toxicidad")
plt.show()

In [ ]:
# Identificar correlaciones más altas
print("🔗 CORRELACIONES MÁS ALTAS:")
print("="*50)

# Obtener correlaciones sin la diagonal
corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_pairs.append({
            'Label1': correlation_matrix.columns[i],
            'Label2': correlation_matrix.columns[j],
            'Correlation': correlation_matrix.iloc[i, j]
        })

# Ordenar por correlación absoluta
corr_df = pd.DataFrame(corr_pairs)
corr_df['Abs_Correlation'] = corr_df['Correlation'].abs()
corr_df = corr_df.sort_values('Abs_Correlation', ascending=False)

print("Top 10 correlaciones más fuertes:")
for _, row in corr_df.head(10).iterrows():
    print(f"  {row['Label1']} ↔ {row['Label2']}: {row['Correlation']:.3f}")

## 4. Análisis de Texto

In [ ]:
# Análisis básico de longitud de texto
print("📝 ANÁLISIS DE LONGITUD DE COMENTARIOS:")
print("="*50)

# Calcular estadísticas de longitud
df['text_length'] = df['Text'].str.len()
df['word_count'] = df['Text'].str.split().str.len()

print("Estadísticas de longitud de caracteres:")
print(df['text_length'].describe())
print("\nEstadísticas de número de palabras:")
print(df['word_count'].describe())

In [ ]:
# Visualización de distribución de longitudes
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Distribución de longitud de caracteres
axes[0,0].hist(df['text_length'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].set_title('📏 Distribución de Longitud de Caracteres')
axes[0,0].set_xlabel('Número de Caracteres')
axes[0,0].set_ylabel('Frecuencia')

# Distribución de número de palabras
axes[0,1].hist(df['word_count'], bins=50, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0,1].set_title('📝 Distribución de Número de Palabras')
axes[0,1].set_xlabel('Número de Palabras')
axes[0,1].set_ylabel('Frecuencia')

# Boxplot de longitud por toxicidad general
# Crear una columna de toxicidad general (si cualquier etiqueta es True)
df['is_toxic'] = df[analysis_labels].any(axis=1)

axes[1,0].boxplot([df[df['is_toxic'] == False]['text_length'], 
                   df[df['is_toxic'] == True]['text_length']], 
                  labels=['No Tóxico', 'Tóxico'])
axes[1,0].set_title('📊 Longitud de Caracteres: Tóxico vs No Tóxico')
axes[1,0].set_ylabel('Número de Caracteres')

# Boxplot de palabras por toxicidad general
axes[1,1].boxplot([df[df['is_toxic'] == False]['word_count'], 
                   df[df['is_toxic'] == True]['word_count']], 
                  labels=['No Tóxico', 'Tóxico'])
axes[1,1].set_title('📊 Número de Palabras: Tóxico vs No Tóxico')
axes[1,1].set_ylabel('Número de Palabras')

plt.tight_layout()
plt.show()

In [ ]:
# Análisis de palabras más frecuentes
from collections import Counter
import re

def clean_text_for_analysis(text):
    """Limpia el texto para análisis de frecuencia"""
    # Convertir a minúsculas y eliminar caracteres especiales
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    return text

print("🔤 ANÁLISIS DE PALABRAS MÁS FRECUENTES:")
print("="*50)

# Palabras más frecuentes en comentarios no tóxicos
non_toxic_texts = df[df['is_toxic'] == False]['Text'].fillna('')
non_toxic_words = []
for text in non_toxic_texts:
    non_toxic_words.extend(clean_text_for_analysis(text).split())

# Palabras más frecuentes en comentarios tóxicos
toxic_texts = df[df['is_toxic'] == True]['Text'].fillna('')
toxic_words = []
for text in toxic_texts:
    toxic_words.extend(clean_text_for_analysis(text).split())

# Filtrar palabras vacías comunes
common_stopwords = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could', 'should', 'may', 'might', 'must', 'can', 'i', 'you', 'he', 'she', 'it', 'we', 'they', 'me', 'him', 'her', 'us', 'them', 'my', 'your', 'his', 'her', 'its', 'our', 'their', 'this', 'that', 'these', 'those'}

non_toxic_words = [word for word in non_toxic_words if word not in common_stopwords and len(word) > 2]
toxic_words = [word for word in toxic_words if word not in common_stopwords and len(word) > 2]

print("Top 15 palabras en comentarios NO TÓXICOS:")
non_toxic_freq = Counter(non_toxic_words).most_common(15)
for word, count in non_toxic_freq:
    print(f"  {word}: {count}")

print("\nTop 15 palabras en comentarios TÓXICOS:")
toxic_freq = Counter(toxic_words).most_common(15)
for word, count in toxic_freq:
    print(f"  {word}: {count}")


## 5. Análisis de Desequilibrio de Clases

In [ ]:
print("⚖️ ANÁLISIS DE DESEQUILIBRIO DE CLASES:")
print("="*50)

# Análisis general de toxicidad
toxic_count = df['is_toxic'].sum()
non_toxic_count = len(df) - toxic_count
toxic_ratio = toxic_count / len(df)

print(f"📊 Comentarios tóxicos: {toxic_count:,} ({toxic_ratio:.2%})")
print(f"📊 Comentarios no tóxicos: {non_toxic_count:,} ({1-toxic_ratio:.2%})")
print(f"📊 Ratio de desequilibrio: {non_toxic_count/toxic_count:.2f}:1")

# Análisis por cada etiqueta (solo las que se van a usar)
print("\n🔍 Desequilibrio por etiqueta (solo etiquetas válidas para análisis):")
for col in analysis_labels:
    positive_count = (df[col] == True).sum()
    negative_count = (df[col] == False).sum()
    ratio = negative_count / positive_count if positive_count > 0 else float('inf')
    print(f"  {col}: {ratio:.2f}:1 (False:True)")

## 6. 🔍 Análisis de Comentarios Multi-etiqueta

In [ ]:
print("🏷️ ANÁLISIS DE COMENTARIOS MULTI-ETIQUETA:")
print("="*50)

# Contar cuántas etiquetas positivas tiene cada comentario
df['num_toxic_labels'] = df[analysis_labels].sum(axis=1)

# Distribución de número de etiquetas por comentario
label_distribution = df['num_toxic_labels'].value_counts().sort_index()
print("Distribución de número de etiquetas tóxicas por comentario:")
for num_labels, count in label_distribution.items():
    percentage = (count / len(df)) * 100
    print(f"  {num_labels} etiquetas: {count:,} comentarios ({percentage:.2f}%)")

In [ ]:
# Visualización de comentarios multi-etiqueta
plt.figure(figsize=(12, 6))

# Gráfico de barras
bars = plt.bar(label_distribution.index, label_distribution.values, 
               color=plt.cm.Spectral(np.linspace(0, 1, len(label_distribution))))

# Añadir etiquetas con porcentajes
for bar, count in zip(bars, label_distribution.values):
    height = bar.get_height()
    percentage = (count / len(df)) * 100
    plt.text(bar.get_x() + bar.get_width()/2., height + 5,
             f'{count}\n({percentage:.1f}%)', 
             ha='center', va='bottom', fontweight='bold')

plt.xlabel('Número de Etiquetas Tóxicas por Comentario', fontsize=12, fontweight='bold')
plt.ylabel('Número de Comentarios', fontsize=12, fontweight='bold')
plt.title('🏷️ Distribución de Comentarios Multi-etiqueta', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Resumen de Insights y Recomendaciones

In [ ]:
print("🔍 INSIGHTS PRINCIPALES DEL EDA:")
print("="*60)

print("\n🔍 1. ESTRUCTURA DE DATOS:")
print(f"   • Dataset con {len(df):,} comentarios y {len(analysis_labels)} etiquetas de toxicidad válidas")
print(f"   • Etiquetas válidas: {', '.join(analysis_labels)}")
if insufficient_labels:
    print(f"   • Etiquetas excluidas por datos insuficientes: {', '.join(insufficient_labels)}")

print("\n⚖️ 2. DESEQUILIBRIO DE CLASES:")
print(f"   • {toxic_ratio:.1%} de comentarios son tóxicos en general")
print("   • Todas las etiquetas muestran fuerte desequilibrio (mayoría de casos negativos)")
print("   • Esto requerirá técnicas de balanceamiento o métricas especiales")

print("\n🔍 3. CORRELACIONES:")
if len(corr_df) > 0:
    strongest_corr = corr_df.iloc[0]
    print(f"   • Correlación más fuerte: {strongest_corr['Label1']} ↔ {strongest_corr['Label2']} ({strongest_corr['Correlation']:.3f})")
print("   • Algunas etiquetas están correlacionadas, considerar esto en el modelado")

print("\n🔍 4. CARACTERÍSTICAS DEL TEXTO:")
print(f"   • Longitud promedio: {df['text_length'].mean():.0f} caracteres")
print(f"   • Número promedio de palabras: {df['word_count'].mean():.1f}")
multi_label_pct = (df['num_toxic_labels'] > 1).sum() / len(df) * 100
print(f"   • {multi_label_pct:.1f}% de comentarios tóxicos tienen múltiples etiquetas")

print("\n🔍 5. RECOMENDACIONES PARA EL MODELADO:")
print("   • Usar métricas apropiadas para datos desequilibrados (F1, ROC-AUC, Precision-Recall)")
print("   • Considerar técnicas de balanceamiento (SMOTE, class_weight, etc.)")
print("   • Implementar validación estratificada")
print("   • Evaluar modelos multi-etiqueta vs binarios independientes")
print("   • Preprocesar texto: lowercasing, eliminación de caracteres especiales, etc.")
print("   • Considerar features de longitud de texto como variables adicionales")
if insufficient_labels:
    print(f"   • Excluir del modelado las etiquetas: {', '.join(insufficient_labels)} por falta de datos")

# Guardar algunas estadísticas clave para uso posterior
stats_summary = {
    'total_comments': len(df),
    'toxic_ratio': toxic_ratio,
    'valid_label_columns': analysis_labels,
    'excluded_labels': insufficient_labels,
    'avg_text_length': df['text_length'].mean(),
    'avg_word_count': df['word_count'].mean(),
    'multi_label_ratio': multi_label_pct / 100
}

print(f"\n🔍 Estadísticas guardadas para referencia futura:")
for key, value in stats_summary.items():
    print(f"   {key}: {value}")

print("\n🔍 EDA COMPLETADO - Listo para la fase de preprocesamiento y modelado! 🔍")